In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train_data = pd.read_json('/content/drive/MyDrive/training_data/train.json').T

In [ ]:
def Faithfulness(original_preds, contrast_preds):
    if len(original_preds) != len(contrast_preds):
        raise ValueError("The lists of predictions must have the same length.")
    faithfulness = sum(abs(o - c) for o, c in zip(original_preds, contrast_preds)) / len(original_preds)
    return faithfulness

def Consistency(original_preds, contrast_preds):
    if len(original_preds) != len(contrast_preds):
        raise ValueError("The lists of predictions must have the same length.")
    consistency = sum(1-abs(o - c) for o, c in zip(original_preds, contrast_preds)) / len(original_preds)
    return consistency

In [ ]:
train_data.head()

,Type,Section_id,Primary_id,Secondary_id,Statement,Label
5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction
86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction
dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment
20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction
f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction


In [ ]:
for i in train_data.columns:
  print(i, ' - ',train_data[i].isna().sum())

Type  -  0
Section_id  -  0
Primary_id  -  0
Secondary_id  -  1035
Statement  -  0
Label  -  0


In [ ]:
import os
import json

In [ ]:
# Path to the folder containing JSON files
json_folder_path = '/content/drive/MyDrive/training_data/CT json'

In [ ]:
# Function to load JSON data
def load_json_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

In [ ]:
import re

In [ ]:
import pandas as pd
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch

In [ ]:
def remove_non_alphabetic(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

In [ ]:
data=pd.read_json('/content/drive/MyDrive/training_data/gold_practice_test.json')

In [ ]:
data = data.T

In [ ]:
data['Intervention'].unique()

array(['Contradiction', 'Paraphrase', 'Text_appended',
       'Numerical_contradiction', nan, 'Numerical_paraphrase'],
      dtype=object)

In [ ]:
contradict_index = []
contradict2=[]
for i in range(0, data.shape[0]):
  if data['Intervention'][i] in ['Contradiction']:
    contradict_index.append(data.index.tolist()[i])
    contradict2.append(data['Causal_type'][i][1])

In [ ]:
original = []
for i in range(data.shape[0]):
    if data['Intervention'][i] in ['Text_appended']:
      if data['Causal_type'][i][1] in contradict2:
        original.append(data.index[i])

In [ ]:
len(original),len(contradict_index)

(600, 600)

In [ ]:
contradict_data = data.loc[contradict_index]

In [ ]:
contradict_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id
14e55d50-c012-4f32-afc5-6f5f7a030667,Single,Adverse Events,NCT00328783,the primary trial documents a comprehensive li...,Contradiction,Contradiction,"[Altering, fef5aedb-21c0-4130-b8a3-cb1df5017c7a]",NaN
64ab5bcd-229a-4aac-8d01-609e514400d7,Single,Intervention,NCT00374322,the patient cohorts in the primary clinical tr...,Contradiction,Contradiction,"[Altering, c40fdf91-9f81-454a-bba9-ec5c950cef6e]",NaN
f0271e34-7791-441b-ad94-a8e2e7f1f909,Single,Results,NCT01644890,cohort 1 of the primary clinical trial had an ...,Contradiction,Contradiction,"[Altering, 8c6314f7-34f5-42a7-ba66-b95d050a6315]",NaN
11073b50-f135-48a1-a8ac-333375a49c14,Single,Eligibility,NCT02186015,children are allowed to participate in the pri...,Contradiction,Contradiction,"[Altering, a805f174-2707-4302-a195-05153ea8523e]",NaN
78ac8091-0f1b-44d1-a760-71bfdc2d46ff,Single,Adverse Events,NCT00191815,none of the participants in the first group of...,Contradiction,Contradiction,"[Preserving, 8e91e404-086e-4063-8d54-83d1b8ebd...",NaN


In [ ]:
original_data=data.loc[original]

In [ ]:
original_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id
6c337426-cff6-4f2c-acab-e7bf15f09cc2,Comparison,Adverse Events,NCT01852032,the primary trial and the secondary trial repo...,Contradiction,Text_appended,"[Preserving, c7a953b0-ac96-4e0a-9c1d-98cf8af28...",NCT01830933
141798cc-9851-4bdf-b8a9-ce688db34e72,Single,Eligibility,NCT00438100,ongoing flucytosine treatments are not permitt...,Entailment,Text_appended,"[Preserving, f10f3b84-e039-4a02-a98b-86d149d9a...",NaN
ec5ca1ce-8399-4213-9f44-53e55b7352db,Comparison,Adverse Events,NCT01216176,child needs help playing with other children i...,Contradiction,Text_appended,"[Preserving, 9c033072-5f94-4d79-af47-126d531f0...",NCT00325234
95f5af8f-1590-4254-bd9f-885b6720b7b5,Single,Eligibility,NCT00375427,Patients must have a Life expectancy of at lea...,Entailment,Text_appended,"[Preserving, 618a3ae1-b22c-4c35-86a6-190c1b5b5...",NaN
41a49fe6-55b7-4f89-bc8c-b0107350a79a,Single,Intervention,NCT00904033,neither of the cohorts in the primary trial ar...,Contradiction,Text_appended,"[Preserving, 63a0a9fc-80b9-450f-b2c6-7aee37ce2...",NaN


In [ ]:
preserving_index = []
preserving2=[]
for i in range(0, data.shape[0]):
  if data['Intervention'][i] in ['Paraphrase']:
    preserving_index.append(data.index.tolist()[i])
    preserving2.append(data['Causal_type'][i][1])

In [ ]:
original_preserving = []
for i in range(data.shape[0]):
    if data['Intervention'][i] in ['Text_appended']:
      if data['Causal_type'][i][1] in preserving2:
        original_preserving.append(data.index[i])

In [ ]:
len(preserving_index), len(original_preserving)

(600, 600)

In [ ]:
original_preserving_data=data.loc[original_preserving]

In [ ]:
original_preserving_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id
6c337426-cff6-4f2c-acab-e7bf15f09cc2,Comparison,Adverse Events,NCT01852032,the primary trial and the secondary trial repo...,Contradiction,Text_appended,"[Preserving, c7a953b0-ac96-4e0a-9c1d-98cf8af28...",NCT01830933
141798cc-9851-4bdf-b8a9-ce688db34e72,Single,Eligibility,NCT00438100,ongoing flucytosine treatments are not permitt...,Entailment,Text_appended,"[Preserving, f10f3b84-e039-4a02-a98b-86d149d9a...",NaN
ec5ca1ce-8399-4213-9f44-53e55b7352db,Comparison,Adverse Events,NCT01216176,child needs help playing with other children i...,Contradiction,Text_appended,"[Preserving, 9c033072-5f94-4d79-af47-126d531f0...",NCT00325234
95f5af8f-1590-4254-bd9f-885b6720b7b5,Single,Eligibility,NCT00375427,Patients must have a Life expectancy of at lea...,Entailment,Text_appended,"[Preserving, 618a3ae1-b22c-4c35-86a6-190c1b5b5...",NaN
41a49fe6-55b7-4f89-bc8c-b0107350a79a,Single,Intervention,NCT00904033,neither of the cohorts in the primary trial ar...,Contradiction,Text_appended,"[Preserving, 63a0a9fc-80b9-450f-b2c6-7aee37ce2...",NaN


In [ ]:
preserving_data=data.loc[preserving_index]

In [ ]:
preserving_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id
ea57b54d-1912-4d9f-bcf7-7f423aed195b,Comparison,Intervention,NCT01572727,"in the primary clinical trial, patients receiv...",Entailment,Paraphrase,"[Preserving, 3ba42f68-a643-4d10-afd0-cbb358d05...",NCT00328783
47f4c385-4dd8-4b6e-a365-bb2fac258ab9,Single,Results,NCT00915603,"in cohort 1 of the primary clinical trial, the...",Contradiction,Paraphrase,"[Preserving, 0cef8c8e-7986-46c7-a597-c5733a989...",NaN
d31f179f-1253-44c8-8fc6-167f7b3d41db,Comparison,Intervention,NCT01572727,the intake of paclitaxel for patients in the p...,Entailment,Paraphrase,"[Preserving, 3ba42f68-a643-4d10-afd0-cbb358d05...",NCT00328783
76236597-7a5e-4042-b1c4-ddd5a4aa57e7,Comparison,Results,NCT00852930,the collection of overall survival metrics of ...,Contradiction,Paraphrase,"[Preserving, dc4c57b0-8c75-494e-81b0-3528df85e...",NCT02915744
572c52ba-9518-4b72-9a32-a08604808764,Single,Results,NCT00357110,the primary clinical trial reported 6 losses i...,Contradiction,Paraphrase,"[Preserving, f114846e-79d8-4712-add0-bb34b65d4...",NaN


In [ ]:
c=0
for i in original:
  if i not in original_preserving:
    c+=1
c

0

In [ ]:
def pre_processing(data,json_folder_path,remove_non_alphabetic):
  data['Primary_id_intervention']=None
  data['Primary_id_Eligibility']=None
  data['Primary_id_results']=None
  data['Primary_id_Adverse_events']=None
  for i in range(0,len(data['Primary_id'])):
      json_file_path = os.path.join(json_folder_path, data['Primary_id'][i] + '.json')
      if os.path.exists(json_file_path):
          json_data = load_json_data(json_file_path)
          data['Primary_id_intervention'][i] = json_data['Intervention']
          data['Primary_id_Eligibility'][i]=json_data['Eligibility']
          data['Primary_id_results'][i]=json_data['Results']
          data['Primary_id_Adverse_events'][i]=json_data['Adverse Events']
      else:
          print(f"File not found: {json_file_path}")
          data['Primary_id_intervention'][i] = None
          data['Primary_id_Eligibility'][i]=None
          data['Primary_id_results'][i]=None
          data['Primary_id_Adverse_events'][i]=None
  data['Secondary_id_intervention']="None"
  data['Secondary_id_Eligibility']="None"
  data['Secondary_id_results']="None"
  data['Secondary_id_Adverse_events']="None"
  for i in range(0,len(data['Secondary_id'])):
      if data['Type'][i]=='Comparison':
        json_file_path = os.path.join(json_folder_path, data['Secondary_id'][i] + '.json')
        if os.path.exists(json_file_path):
          json_data = load_json_data(json_file_path)
          data['Secondary_id_intervention'][i] = json_data['Intervention']
          data['Secondary_id_Eligibility'][i]=json_data['Eligibility']
          data['Secondary_id_results'][i]=json_data['Results']
          data['Secondary_id_Adverse_events'][i]=json_data['Adverse Events']
        else:
          data['Secondary_id_intervention'][i] = "None"
          data['Secondary_id_Eligibility'][i]="None"
          data['Secondary_id_results'][i]="None"
          data['Secondary_id_Adverse_events'][i]="None"
  for i in range(0,data.shape[0]):
    data1=data['Primary_id_intervention'][i]
    joined_string = ' '.join(data1)
    data['Primary_id_intervention'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
    data1=data['Primary_id_Eligibility'][i]
    joined_string = ' '.join(data1)
    data['Primary_id_Eligibility'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
    data1=data['Primary_id_results'][i]
    joined_string = ' '.join(data1)
    data['Primary_id_results'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
    data1=data['Primary_id_Adverse_events'][i]
    joined_string = ' '.join(data1)
    data['Primary_id_Adverse_events'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
    data1=data['Secondary_id_intervention'][i]
    joined_string = ' '.join(data1)
    data['Secondary_id_intervention'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
    data1=data['Secondary_id_Eligibility'][i]
    joined_string = ' '.join(data1)
    data['Secondary_id_Eligibility'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
    data1=data['Secondary_id_results'][i]
    joined_string = ' '.join(data1)
    data['Secondary_id_results'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
    data1=data['Secondary_id_Adverse_events'][i]
    joined_string = ' '.join(data1)
    data['Secondary_id_Adverse_events'][i] = re.sub(r'[^a-zA-Z0-9\s]', '', joined_string)
  for i in range(0,data.shape[0]):
    if data['Secondary_id_intervention'][i]=="N o n e":
      data['Secondary_id_Adverse_events'][i]="None"
      data['Secondary_id_Eligibility'][i]="None"
      data['Secondary_id_intervention'][i]="None"
      data['Secondary_id_results'][i]="None"
  data['Concatenated_Text'] = "None"
  for i in range(data.shape[0]):
      if data['Section_id'][i] == "Intervention":
          primary_id = str(data['Primary_id_intervention'][i])
          secondary_id = str(data['Secondary_id_intervention'][i])
          statement = str(data['Statement'][i])
          data['Concatenated_Text'][i] = 'The primary id intervention is ' + primary_id + ' and the secondary id intervention is ' + secondary_id + ' and the Statement is ' + statement
      elif data['Section_id'][i] == 'Eligibility':
          primary_id = str(data['Primary_id_Eligibility'][i])
          secondary_id = str(data['Secondary_id_Eligibility'][i])
          statement = str(data['Statement'][i])
          data['Concatenated_Text'][i] = 'The primary id eligibility is ' + primary_id + ' and the secondary id eligibility is ' + secondary_id + ' and the Statement is ' + statement
      elif data['Section_id'][i] == 'Adverse Events':
          primary_id = str(data['Primary_id_Adverse_events'][i])
          secondary_id = str(data['Secondary_id_Adverse_events'][i])
          statement = str(data['Statement'][i])
          data['Concatenated_Text'][i] = 'The primary id adverse events is ' + primary_id + ' and the secondary id adverse events is ' + secondary_id + ' and the Statement is ' + statement
      elif data['Section_id'][i] == 'Results':
          primary_id = str(data['Primary_id_results'][i])
          secondary_id = str(data['Secondary_id_results'][i])
          statement = str(data['Statement'][i])
          data['Concatenated_Text'][i] = 'The primary id results is ' + primary_id + ' and the secondary id results is ' + secondary_id + ' and the Statement is ' + statement

  data['Concatenated_Text'] = data['Concatenated_Text'].astype('string')
  data['Concatenated_Text'] = data['Concatenated_Text'].apply(remove_non_alphabetic)
  data['Concatenated_Text'] = data['Concatenated_Text'].str.lower()
  return data

In [ ]:
validation_data=pd.read_json('/content/drive/MyDrive/training_data/dev.json')
validation_data=validation_data.T

In [ ]:
gold_data = pd.read_json('/content/drive/MyDrive/training_data/gold_practice_test.json').T

In [ ]:
test_data=pd.read_json('/content/drive/MyDrive/training_data/test.json').T

In [ ]:
train_data=pre_processing(train_data,json_folder_path,remove_non_alphabetic)
validation_data=pre_processing(validation_data,json_folder_path,remove_non_alphabetic)
contradict_data=pre_processing(contradict_data,json_folder_path,remove_non_alphabetic)
original_data=pre_processing(original_data,json_folder_path,remove_non_alphabetic)
preserving_data=pre_processing(preserving_data,json_folder_path,remove_non_alphabetic)
gold_data = pre_processing(gold_data,json_folder_path,remove_non_alphabetic)

In [ ]:
test_data = pre_processing(test_data,json_folder_path,remove_non_alphabetic)

In [ ]:
train_data.head()

,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_id_intervention,Primary_id_Eligibility,Primary_id_results,Primary_id_Adverse_events,Secondary_id_intervention,Secondary_id_Eligibility,Secondary_id_results,Secondary_id_Adverse_events,Concatenated_Text
5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction,INTERVENTION 1 Diagnostic FLT PET Patient...,Inclusion Criteria A new diagnosis of invasi...,Outcome Measurement Percent Change in Net I...,Adverse Events 1 Total 028 000,INTERVENTION 1 Arm A Patients receive ora...,Inclusion Criteria Histologically confirmed ...,Outcome Measurement Progressionfree Surviva...,Adverse Events 1 Total 419 2105 Hemoglobin...,the primary id intervention is intervention 1 ...
86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction,INTERVENTION 1 Paclitaxel Gemcitabine Bev...,DISEASE CHARACTERISTICS Histologically or cy...,Outcome Measurement 6month Progressionfree ...,Adverse Events 1 Total 2049 4082 Febrile n...,None,None,None,None,the primary id eligibility is disease characte...
dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment,INTERVENTION 1 Albuminbound Paclitaxel Carb...,Inclusion Criteria Confirmed adenocarcinoma ...,Outcome Measurement Percentage of Participa...,Adverse Events 1 Total 532 1563 Febrile ne...,INTERVENTION 1 Ramucirumab IMC1121B Doceta...,Inclusion Criteria Participant is able to pr...,Outcome Measurement ProgressionFree Surviva...,Adverse Events 1 Total 285752 3790 Anaemia...,the primary id adverse events is adverse event...
20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction,INTERVENTION 1 Ixabepilone Brand name is ...,Inclusion Criteria Patients with histologic ...,Outcome Measurement Complete Response Rate ...,Adverse Events 1 Total 215 1333 Neutropeni...,None,None,None,None,the primary id eligibility is inclusion criter...
f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction,INTERVENTION 1 Laser Therapy Alone therap...,Inclusion Criteria Breast cancer survivors w...,Outcome Measurement LDex Change Bioimpeda...,Adverse Events 1 Total 015 000 Adverse Event...,INTERVENTION 1 Part A Abemaciclib HR HER2 B...,Inclusion Criteria Have brain metastases sec...,Outcome Measurement Percentage of Participa...,Adverse Events 1 Total 627 2222 Anaemia 02...,the primary id intervention is intervention 1 ...


In [ ]:
for i in train_data.columns:
  print(i, ' - ',train_data[i].isna().sum())

Type  -  0
Section_id  -  0
Primary_id  -  0
Secondary_id  -  1035
Statement  -  0
Label  -  0
Primary_id_intervention  -  0
Primary_id_Eligibility  -  0
Primary_id_results  -  0
Primary_id_Adverse_events  -  0
Secondary_id_intervention  -  0
Secondary_id_Eligibility  -  0
Secondary_id_results  -  0
Secondary_id_Adverse_events  -  0
Concatenated_Text  -  0


In [ ]:
gold_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id,Primary_id_intervention,Primary_id_Eligibility,Primary_id_results,Primary_id_Adverse_events,Secondary_id_intervention,Secondary_id_Eligibility,Secondary_id_results,Secondary_id_Adverse_events,Concatenated_Text
14e55d50-c012-4f32-afc5-6f5f7a030667,Single,Adverse Events,NCT00328783,the primary trial documents a comprehensive li...,Contradiction,Contradiction,"[Altering, fef5aedb-21c0-4130-b8a3-cb1df5017c7a]",NaN,INTERVENTION 1 Active Breathing Coordinator...,Inclusion Criteria Requiring adjuvant or pos...,Outcome Measurement Dosimetric Evaluation M...,Adverse Events 1 Total 00,None,None,None,None,the primary id adverse events is adverse event...
ea57b54d-1912-4d9f-bcf7-7f423aed195b,Comparison,Intervention,NCT01572727,"in the primary clinical trial, patients receiv...",Entailment,Paraphrase,"[Preserving, 3ba42f68-a643-4d10-afd0-cbb358d05...",NCT00328783,INTERVENTION 1 BKM120 and Paclitaxel Adul...,Inclusion Criteria Breast cancer that is loc...,Outcome Measurement Progressionfree Surviva...,Adverse Events 1 Total 61202 3020 FEBRILE ...,INTERVENTION 1 Active Breathing Coordinator...,Inclusion Criteria Requiring adjuvant or pos...,Outcome Measurement Dosimetric Evaluation M...,Adverse Events 1 Total 00,the primary id intervention is intervention 1 ...
6c337426-cff6-4f2c-acab-e7bf15f09cc2,Comparison,Adverse Events,NCT01852032,the primary trial and the secondary trial repo...,Contradiction,Text_appended,"[Preserving, c7a953b0-ac96-4e0a-9c1d-98cf8af28...",NCT01830933,INTERVENTION 1 Breast Cancer Patients Tom...,Inclusion Criteria 35 years of age or older ...,Outcome Measurement Beta of CT Coronal View...,Adverse Events 1 Total 023 000,INTERVENTION 1 Usual Care Usual Care is t...,Inclusion Criteria Patient component Women...,Outcome Measurement Knowledge of Breast Can...,Adverse Events 1 Total 0655 000 Adverse Even...,the primary id adverse events is adverse event...
141798cc-9851-4bdf-b8a9-ce688db34e72,Single,Eligibility,NCT00438100,ongoing flucytosine treatments are not permitt...,Entailment,Text_appended,"[Preserving, f10f3b84-e039-4a02-a98b-86d149d9a...",NaN,INTERVENTION 1 Capecitabine Arm Capecitab...,Inclusion Criteria Biopsydiagnosed breast ca...,Outcome Measurement Progression Free Surviv...,Adverse Events 1 Total 071 000 Adverse Event...,None,None,None,None,the primary id eligibility is inclusion criter...
fdbb037a-8084-4267-9143-f989d26726ab,Single,Results,NCT00217399,all the primary trial patients had a minimum o...,Contradiction,Numerical_contradiction,"[Preserving, 985bc5a0-bc21-447f-92f1-150e23ede...",NaN,INTERVENTION 1 Sorafenib and Anastrozole ...,Inclusion Criteria Histologically or cytolog...,Outcome Measurement Complete Response Part...,Adverse Events 1 Total 1235 3429 handfoot ...,None,None,None,None,the primary id results is outcome measurement ...


In [ ]:
validation_data.head()

,Type,Section_id,Primary_id,Statement,Label,Secondary_id,Primary_id_intervention,Primary_id_Eligibility,Primary_id_results,Primary_id_Adverse_events,Secondary_id_intervention,Secondary_id_Eligibility,Secondary_id_results,Secondary_id_Adverse_events,Concatenated_Text
1adc970c-d433-44d0-aa09-d3834986f7a2,Single,Results,NCT00066573,there is a 13.2% difference between the result...,Contradiction,NaN,INTERVENTION 1 Exemestane Patients receiv...,DISEASE CHARACTERISTICS Histologically confi...,Outcome Measurement Eventfree Survival Ev...,Adverse Events 1 Total 193761 051 Cardiac ...,None,None,None,None,the primary id results is outcome measurement ...
6b9162d0-0816-46d4-81af-c60028dcc63b,Comparison,Eligibility,NCT00425854,Patients with significantly elevated ejection ...,Contradiction,NCT01224678,INTERVENTION 1 Cohort B Patients with HER...,Inclusion criteria Inclusion Criteria Female...,Outcome Measurement Objective Response OR ...,Adverse Events 1 Total 1329 4483 Anaemia 1...,INTERVENTION 1 Placebo Patients receive o...,Premenopausal women 55 years of age or younger...,Outcome Measurement Percent Change Between ...,Adverse Events 1 Total 0150 000 Adverse Even...,the primary id eligibility is inclusion criter...
0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f,Comparison,Adverse Events,NCT02273973,a significant number of the participants in th...,Contradiction,NCT00281697,INTERVENTION 1 Experimental Taselisib Letr...,Inclusion Criteria Female participants Pos...,Outcome Measurement Percentage of Participa...,Adverse Events 1 Total 20167 1198 Cardiac ...,INTERVENTION 1 Standard Chemotherapy Bevac...,Inclusion Criteria Signed informed consent f...,Outcome Measurement Progressionfree Surviva...,Adverse Events 1 Total 112458 2445 Febrile...,the primary id adverse events is adverse event...
cc1f712a-2116-4e40-9810-f315e3fa5ff8,Single,Results,NCT00593346,the primary trial does not report the PFS or o...,Entailment,NaN,INTERVENTION 1 Accelerated Partial Breast B...,Inclusion Criteria AJCC stage 0 I or II TisN...,Outcome Measurement Local Control Using Ips...,Adverse Events 1 Total 0151 000,None,None,None,None,the primary id results is outcome measurement ...
904061c0-14fa-4f13-9118-9a41e24fa8eb,Single,Eligibility,NCT02340221,Prior treatment with fulvestrant or with a pho...,Contradiction,NaN,INTERVENTION 1 PlaceboFulvestrant Partici...,Inclusion Criteria Postmenopausal women with...,Outcome Measurement ProgressionFree Surviva...,Adverse Events 1 Total 19213 892 Pancytope...,None,None,None,None,the primary id eligibility is inclusion criter...


In [ ]:
contradict_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id,Primary_id_intervention,Primary_id_Eligibility,Primary_id_results,Primary_id_Adverse_events,Secondary_id_intervention,Secondary_id_Eligibility,Secondary_id_results,Secondary_id_Adverse_events,Concatenated_Text
14e55d50-c012-4f32-afc5-6f5f7a030667,Single,Adverse Events,NCT00328783,the primary trial documents a comprehensive li...,Contradiction,Contradiction,"[Altering, fef5aedb-21c0-4130-b8a3-cb1df5017c7a]",NaN,INTERVENTION 1 Active Breathing Coordinator...,Inclusion Criteria Requiring adjuvant or pos...,Outcome Measurement Dosimetric Evaluation M...,Adverse Events 1 Total 00,None,None,None,None,the primary id adverse events is adverse event...
64ab5bcd-229a-4aac-8d01-609e514400d7,Single,Intervention,NCT00374322,the patient cohorts in the primary clinical tr...,Contradiction,Contradiction,"[Altering, c40fdf91-9f81-454a-bba9-ec5c950cef6e]",NaN,INTERVENTION 1 Lapatinib 1500 mg Particip...,Inclusion Criteria Have histologically or cy...,Outcome Measurement Number of Participants ...,Adverse Events 1 Total 991573 629 Neutrope...,None,None,None,None,the primary id intervention is intervention 1 ...
f0271e34-7791-441b-ad94-a8e2e7f1f909,Single,Results,NCT01644890,cohort 1 of the primary clinical trial had an ...,Contradiction,Contradiction,"[Altering, 8c6314f7-34f5-42a7-ba66-b95d050a6315]",NaN,INTERVENTION 1 NK105 received NK105 65 mg...,Inclusion Criteria Written informed consent ...,Outcome Measurement Progression Free Surviv...,Adverse Events 1 Total 34214 1589 Leukocyt...,None,None,None,None,the primary id results is outcome measurement ...
11073b50-f135-48a1-a8ac-333375a49c14,Single,Eligibility,NCT02186015,children are allowed to participate in the pri...,Contradiction,Contradiction,"[Altering, a805f174-2707-4302-a195-05153ea8523e]",NaN,INTERVENTION 1 Cholecalciferol Enrolled w...,Inclusion Criteria Metastatic breast cancer ...,Outcome Measurement Change in Serum 25OHD ...,Adverse Events 1 Total 011 000 Adverse Event...,None,None,None,None,the primary id eligibility is inclusion criter...
78ac8091-0f1b-44d1-a760-71bfdc2d46ff,Single,Adverse Events,NCT00191815,none of the participants in the first group of...,Contradiction,Contradiction,"[Preserving, 8e91e404-086e-4063-8d54-83d1b8ebd...",NaN,INTERVENTION 1 Gemcitabine Cisplatin Gem...,Inclusion Criteria You are female in the age...,Outcome Measurement Objective Tumor Respons...,Adverse Events 1 Total 6 Atrial fibrillati...,None,None,None,None,the primary id adverse events is adverse event...


In [ ]:
original_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id,Primary_id_intervention,Primary_id_Eligibility,Primary_id_results,Primary_id_Adverse_events,Secondary_id_intervention,Secondary_id_Eligibility,Secondary_id_results,Secondary_id_Adverse_events,Concatenated_Text
6c337426-cff6-4f2c-acab-e7bf15f09cc2,Comparison,Adverse Events,NCT01852032,the primary trial and the secondary trial repo...,Contradiction,Text_appended,"[Preserving, c7a953b0-ac96-4e0a-9c1d-98cf8af28...",NCT01830933,INTERVENTION 1 Breast Cancer Patients Tom...,Inclusion Criteria 35 years of age or older ...,Outcome Measurement Beta of CT Coronal View...,Adverse Events 1 Total 023 000,INTERVENTION 1 Usual Care Usual Care is t...,Inclusion Criteria Patient component Women...,Outcome Measurement Knowledge of Breast Can...,Adverse Events 1 Total 0655 000 Adverse Even...,the primary id adverse events is adverse event...
141798cc-9851-4bdf-b8a9-ce688db34e72,Single,Eligibility,NCT00438100,ongoing flucytosine treatments are not permitt...,Entailment,Text_appended,"[Preserving, f10f3b84-e039-4a02-a98b-86d149d9a...",NaN,INTERVENTION 1 Capecitabine Arm Capecitab...,Inclusion Criteria Biopsydiagnosed breast ca...,Outcome Measurement Progression Free Surviv...,Adverse Events 1 Total 071 000 Adverse Event...,None,None,None,None,the primary id eligibility is inclusion criter...
ec5ca1ce-8399-4213-9f44-53e55b7352db,Comparison,Adverse Events,NCT01216176,child needs help playing with other children i...,Contradiction,Text_appended,"[Preserving, 9c033072-5f94-4d79-af47-126d531f0...",NCT00325234,INTERVENTION 1 Phase 1 Cohort A Dual tre...,Inclusion Criteria Phase 1 Cohort A Female ...,Outcome Measurement Phase I Cohort A Maximu...,Adverse Events 1 Total 312 2500 Atrial fib...,INTERVENTION 1 PemetrexedCarboplatin Peme...,Inclusion Criteria Females with histologic o...,Outcome Measurement Tumor Response Rate P...,Adverse Events 1 Total 1865 2769 Anaemia 5...,the primary id adverse events is adverse event...
95f5af8f-1590-4254-bd9f-885b6720b7b5,Single,Eligibility,NCT00375427,Patients must have a Life expectancy of at lea...,Entailment,Text_appended,"[Preserving, 618a3ae1-b22c-4c35-86a6-190c1b5b5...",NaN,INTERVENTION 1 Zoledronic Acid Every 3 Mont...,Inclusion criteria Female patients 18 years...,Outcome Measurement Annual Overall Skeletal...,Adverse Events 1 Total 21209 1005 Anaemia ...,None,None,None,None,the primary id eligibility is inclusion criter...
41a49fe6-55b7-4f89-bc8c-b0107350a79a,Single,Intervention,NCT00904033,neither of the cohorts in the primary trial ar...,Contradiction,Text_appended,"[Preserving, 63a0a9fc-80b9-450f-b2c6-7aee37ce2...",NaN,INTERVENTION 1 No Exercise Multivitamin A...,Inclusion Criteria Must be female Women of...,Outcome Measurement Bone Resorption Exercis...,Adverse Events 1 Total 010 000 Adverse Event...,None,None,None,None,the primary id intervention is intervention 1 ...


In [ ]:
preserving_data.head()

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id,Primary_id_intervention,Primary_id_Eligibility,Primary_id_results,Primary_id_Adverse_events,Secondary_id_intervention,Secondary_id_Eligibility,Secondary_id_results,Secondary_id_Adverse_events,Concatenated_Text
ea57b54d-1912-4d9f-bcf7-7f423aed195b,Comparison,Intervention,NCT01572727,"in the primary clinical trial, patients receiv...",Entailment,Paraphrase,"[Preserving, 3ba42f68-a643-4d10-afd0-cbb358d05...",NCT00328783,INTERVENTION 1 BKM120 and Paclitaxel Adul...,Inclusion Criteria Breast cancer that is loc...,Outcome Measurement Progressionfree Surviva...,Adverse Events 1 Total 61202 3020 FEBRILE ...,INTERVENTION 1 Active Breathing Coordinator...,Inclusion Criteria Requiring adjuvant or pos...,Outcome Measurement Dosimetric Evaluation M...,Adverse Events 1 Total 00,the primary id intervention is intervention 1 ...
47f4c385-4dd8-4b6e-a365-bb2fac258ab9,Single,Results,NCT00915603,"in cohort 1 of the primary clinical trial, the...",Contradiction,Paraphrase,"[Preserving, 0cef8c8e-7986-46c7-a597-c5733a989...",NaN,INTERVENTION 1 PaclitaxelBevacizumabEveroli...,Inclusion Criteria Female or male patients 1...,Outcome Measurement ProgressionFree Surviva...,Adverse Events 1 Total 1655 2909 NEUTROPEN...,None,None,None,None,the primary id results is outcome measurement ...
d31f179f-1253-44c8-8fc6-167f7b3d41db,Comparison,Intervention,NCT01572727,the intake of paclitaxel for patients in the p...,Entailment,Paraphrase,"[Preserving, 3ba42f68-a643-4d10-afd0-cbb358d05...",NCT00328783,INTERVENTION 1 BKM120 and Paclitaxel Adul...,Inclusion Criteria Breast cancer that is loc...,Outcome Measurement Progressionfree Surviva...,Adverse Events 1 Total 61202 3020 FEBRILE ...,INTERVENTION 1 Active Breathing Coordinator...,Inclusion Criteria Requiring adjuvant or pos...,Outcome Measurement Dosimetric Evaluation M...,Adverse Events 1 Total 00,the primary id intervention is intervention 1 ...
76236597-7a5e-4042-b1c4-ddd5a4aa57e7,Comparison,Results,NCT00852930,the collection of overall survival metrics of ...,Contradiction,Paraphrase,"[Preserving, dc4c57b0-8c75-494e-81b0-3528df85e...",NCT02915744,INTERVENTION 1 Laser Therapy Alone therap...,Inclusion Criteria Breast cancer survivors w...,Outcome Measurement LDex Change Bioimpeda...,Adverse Events 1 Total 015 000 Adverse Event...,INTERVENTION 1 NKTR102 In Group A NKTR102...,Inclusion Criteria Female or male age 18 ye...,Outcome Measurement Overall Survival OS of ...,Adverse Events 1 Total 3390 3667 Febrile N...,the primary id results is outcome measurement ...
572c52ba-9518-4b72-9a32-a08604808764,Single,Results,NCT00357110,the primary clinical trial reported 6 losses i...,Contradiction,Paraphrase,"[Preserving, f114846e-79d8-4712-add0-bb34b65d4...",NaN,INTERVENTION 1 Fulvestrant Anastrozole f...,Inclusion Criteria Postmenopausal women with...,Outcome Measurement Patients Eventfree at 1...,Adverse Events 1 Total 16 1667 Goitre 06 0...,None,None,None,None,the primary id results is outcome measurement ...


In [ ]:
test_data.head()

,Type,Section_id,Primary_id,Secondary_id,Statement,Primary_id_intervention,Primary_id_Eligibility,Primary_id_results,Primary_id_Adverse_events,Secondary_id_intervention,Secondary_id_Eligibility,Secondary_id_results,Secondary_id_Adverse_events,Concatenated_Text
47468741-3624-4321-90ee-1c680a293b25,Comparison,Adverse Events,NCT00687102,NCT00819182,the record shows 5 total cases of asthenia and...,INTERVENTION 1 Star Participants Assigned t...,Inclusion Criteria Women enrolled in STAR tr...,Outcome Measurement Mean Change From Baseli...,Adverse Events 1 Total 0733 000 Adverse Even...,INTERVENTION 1 Paced Respiration The pace...,Inclusion Criteria At least 18 years of age ...,Outcome Measurement Hot Flash Frequency P...,Adverse Events 1 Total 088 000 Adverse Event...,the primary id adverse events is adverse event...
93e6ed2b-1e3f-4cbe-a49e-cbc74b31e1aa,Single,Eligibility,NCT01772004,NaN,'the primary clinical trial is open for partic...,INTERVENTION 1 Dose Escalation Cohort Avelu...,Inclusion Criteria for dose escalation and exp...,Outcome Measurement Dose Escalation Cohort ...,Adverse Events 1 Total 34 7500 Anaemia 04...,None,None,None,None,the primary id eligibility is inclusion criter...
ad9fe1aa-9798-4c53-ab23-23ac2924918c,Single,Intervention,NCT00118157,NaN,patients participating in the primary trial re...,INTERVENTION 1 Arm 1 Patients receive ora...,Inclusion Criteria Primary adenocarcinoma of...,Outcome Measurement Tumor Response Rate Com...,Adverse Events 1 Total 1919 10000 Hemolysi...,None,None,None,None,the primary id intervention is intervention 1 ...
cdcb40f6-994c-4085-869a-dab87e67ba04,Single,Results,NCT01106898,NaN,"in the primary clinical trial, 97% of patients...",INTERVENTION 1 Treatment Chemotherapy With ...,Inclusion Criteria Histologically confirmed ...,Outcome Measurement Recurrencefree Survival...,Adverse Events 1 Total 16102 1569 Febrile ...,None,None,None,None,the primary id results is outcome measurement ...
7df66c3f-de87-4542-9797-06325684db65,Comparison,Adverse Events,NCT02002533,NCT02734979,the primary trial has 4 more adverse events re...,INTERVENTION 1 All Participants All peopl...,Inclusion Criteria Newly diagnosed breast ca...,Outcome Measurement Percentage of Eligible ...,Adverse Events 1 Total 034 000,INTERVENTION 1 Biobridge and Lymph Node Tra...,Inclusion Criteria The subject must be a bre...,Outcome Measurement Change in Limb Volume ...,Adverse Events 1 Total 01 000,the primary id adverse events is adverse event...


In [ ]:
def embeddings(data, tokenizer):
  inputs = tokenizer(data['Concatenated_Text'].tolist(), padding=True, truncation=True, return_tensors="pt")
  label_mapping = {'Contradiction': 0, 'Entailment': 1}
  labels = data['Label'].replace(label_mapping).values
  labels = torch.tensor(labels)
  train_inputs=inputs['input_ids']
  train_labels=labels
  train_masks=inputs['attention_mask']
  return train_inputs, train_labels, train_masks

In [ ]:
import torch
from transformers import AutoTokenizer

def preprocess_data(data, tokenizer):
    inputs = tokenizer(data['Concatenated_Text'].tolist(), padding=True, truncation=True, return_tensors="pt")
    train_inputs = inputs['input_ids']
    train_masks = inputs['attention_mask']

    return train_inputs, train_masks


In [ ]:
label_mapping = {'Contradiction': 0, 'Entailment': 1}

# Ensemble learning

Ensemble learning with logistic regression

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer_medbert = AutoTokenizer.from_pretrained("GerMedBERT/medbert-512-no-duplicates")
model_medbert = AutoModelForSequenceClassification.from_pretrained("GerMedBERT/medbert-512-no-duplicates")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/734k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GerMedBERT/medbert-512-no-duplicates and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_medbert.to(device)
model_medbert.load_state_dict(torch.load('/content/drive/MyDrive/model_bert_weights.pth'))
model_medbert.to('cpu')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer_medroberta = AutoTokenizer.from_pretrained("UMCU/MedRoBERTa.nl_NegationDetection")
model_medroberta = AutoModelForSequenceClassification.from_pretrained("UMCU/MedRoBERTa.nl_NegationDetection")


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/893k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/539k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at UMCU/MedRoBERTa.nl_NegationDetection and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_medroberta.to(device)
model_medroberta.load_state_dict(torch.load('/content/drive/MyDrive/model_roberta_weights.pth'))
model_medroberta.to('cpu')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer_medlongformer = AutoTokenizer.from_pretrained("nbroad/longformer-base-health-fact")
model_medlongformer = AutoModelForSequenceClassification.from_pretrained("nbroad/longformer-base-health-fact")

tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of the model checkpoint at nbroad/longformer-base-health-fact were not used when initializing LongformerForSequenceClassification: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_medlongformer.to(device)
model_medlongformer.load_state_dict(torch.load('/content/drive/MyDrive/model_long_weights.pth'))
model_medlongformer.to('cpu')

In [ ]:
tokenizer_medbert.model_max_length=512
tokenizer_medroberta.model_max_length=512
tokenizer_medlongformer.model_max_length=512

In [ ]:
train_inputs_medbert, train_labels_medbert, train_masks_medbert=embeddings(train_data, tokenizer_medbert)
val_inputs_medbert, val_labels_medbert, val_masks_medbert=embeddings(validation_data, tokenizer_medbert)
contradict_inputs_medbert, contradict_labels_medbert, contradict_masks_medbert=embeddings(contradict_data, tokenizer_medbert)
original_inputs_medbert, original_labels_medbert, original_masks_medbert=embeddings(original_data, tokenizer_medbert)
preserving_inputs_medbert, preserving_labels_medbert, preserving_masks_medbert=embeddings(preserving_data, tokenizer_medbert)
gold_inputs_medbert, gold_labels_medbert, gold_masks_medbert = embeddings(gold_data, tokenizer_medbert)
test_inputs_medbert, test_masks_medbert=preprocess_data(test_data, tokenizer_medbert)

In [ ]:
train_inputs_medroberta, train_labels_medroberta, train_masks_medroberta=embeddings(train_data, tokenizer_medroberta)
val_inputs_medroberta, val_labels_medroberta, val_masks_medroberta=embeddings(validation_data, tokenizer_medroberta)
contradict_inputs_medroberta, contradict_labels_medroberta, contradict_masks_medroberta=embeddings(contradict_data, tokenizer_medroberta)
original_inputs_medroberta, original_labels_medroberta, original_masks_medroberta=embeddings(original_data, tokenizer_medroberta)
preserving_inputs_medroberta, preserving_labels_medroberta, preserving_masks_medroberta=embeddings(preserving_data, tokenizer_medroberta)
gold_inputs_medroberta, gold_labels_medroberta, gold_masks_medroberta = embeddings(gold_data, tokenizer_medroberta)
test_inputs_medroberta, test_masks_medroberta=preprocess_data(test_data, tokenizer_medroberta)

In [ ]:
train_inputs_medlongformer, train_labels_medlongformer, train_masks_medlongformer=embeddings(train_data, tokenizer_medlongformer)
val_inputs_medlongformer, val_labels_medlongformer, val_masks_medlongformer=embeddings(validation_data, tokenizer_medlongformer)
contradict_inputs_medlongformer, contradict_labels_medlongformer, contradict_masks_medlongformer=embeddings(contradict_data, tokenizer_medlongformer)
original_inputs_medlongformer, original_labels_medlongformer, original_masks_medlongformer=embeddings(original_data, tokenizer_medlongformer)
preserving_inputs_medlongformer, preserving_labels_medlongformer, preserving_masks_medlongformer=embeddings(preserving_data, tokenizer_medlongformer)
gold_inputs_medlongformer, gold_labels_medlongformer, gold_masks_medlongformer = embeddings(gold_data, tokenizer_medlongformer)
test_inputs_medlongformer, test_masks_medlongformer=preprocess_data(test_data, tokenizer_medlongformer)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.optim as optim

batch_size = 1

train_data_t_medbert = TensorDataset(train_inputs_medbert, train_masks_medbert, train_labels_medbert)
train_sampler_medbert = SequentialSampler(train_data_t_medbert)
train_dataloader_medbert = DataLoader(train_data_t_medbert, sampler=train_sampler_medbert, batch_size=batch_size)

validation_data_t_medbert = TensorDataset(val_inputs_medbert, val_masks_medbert, val_labels_medbert)
validation_sampler_medbert = SequentialSampler(validation_data_t_medbert)
validation_dataloader_medbert = DataLoader(validation_data_t_medbert, sampler=validation_sampler_medbert, batch_size=batch_size)

contradict_data_t_medbert = TensorDataset(contradict_inputs_medbert, contradict_masks_medbert, contradict_labels_medbert)
contradict_sampler_medbert = SequentialSampler(contradict_data_t_medbert)
contradict_dataloader_medbert = DataLoader(contradict_data_t_medbert, sampler=contradict_sampler_medbert, batch_size=batch_size)

original_data_t_medbert = TensorDataset(original_inputs_medbert, original_masks_medbert, original_labels_medbert)
original_sampler_medbert = SequentialSampler(original_data_t_medbert)
original_dataloader_medbert = DataLoader(original_data_t_medbert, sampler=original_sampler_medbert, batch_size=batch_size)

preserving_data_t_medbert = TensorDataset(preserving_inputs_medbert, preserving_masks_medbert, preserving_labels_medbert)
preserving_sampler_medbert = SequentialSampler(preserving_data_t_medbert)
preserving_dataloader_medbert = DataLoader(preserving_data_t_medbert, sampler=preserving_sampler_medbert, batch_size=batch_size)

gold_data_t_medbert = TensorDataset(gold_inputs_medbert, gold_masks_medbert, gold_labels_medbert)
gold_sampler_medbert = SequentialSampler(gold_data_t_medbert)
gold_dataloader_medbert = DataLoader(gold_data_t_medbert, sampler=gold_sampler_medbert, batch_size=batch_size)

test_data_t_medbert = TensorDataset(test_inputs_medbert, test_masks_medbert)
test_dataloader_medbert = DataLoader(test_data_t_medbert, batch_size=batch_size)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.optim as optim

batch_size = 1

train_data_t_medroberta = TensorDataset(train_inputs_medroberta, train_masks_medroberta, train_labels_medroberta)
train_sampler_medroberta = SequentialSampler(train_data_t_medroberta)
train_dataloader_medroberta = DataLoader(train_data_t_medroberta, sampler=train_sampler_medroberta, batch_size=batch_size)

validation_data_t_medroberta = TensorDataset(val_inputs_medroberta, val_masks_medroberta, val_labels_medroberta)
validation_sampler_medroberta = SequentialSampler(validation_data_t_medroberta)
validation_dataloader_medroberta = DataLoader(validation_data_t_medroberta, sampler=validation_sampler_medroberta, batch_size=batch_size)

contradict_data_t_medroberta = TensorDataset(contradict_inputs_medroberta, contradict_masks_medroberta, contradict_labels_medroberta)
contradict_sampler_medroberta = SequentialSampler(contradict_data_t_medroberta)
contradict_dataloader_medroberta = DataLoader(contradict_data_t_medroberta, sampler=contradict_sampler_medroberta, batch_size=batch_size)

original_data_t_medroberta = TensorDataset(original_inputs_medroberta, original_masks_medroberta, original_labels_medroberta)
original_sampler_medroberta = SequentialSampler(original_data_t_medroberta)
original_dataloader_medroberta = DataLoader(original_data_t_medroberta, sampler=original_sampler_medroberta, batch_size=batch_size)

preserving_data_t_medroberta = TensorDataset(preserving_inputs_medroberta, preserving_masks_medroberta, preserving_labels_medroberta)
preserving_sampler_medroberta = SequentialSampler(preserving_data_t_medroberta)
preserving_dataloader_medroberta = DataLoader(preserving_data_t_medroberta, sampler=preserving_sampler_medroberta, batch_size=batch_size)

gold_data_t_medroberta = TensorDataset(gold_inputs_medroberta, gold_masks_medroberta, gold_labels_medroberta)
gold_sampler_medroberta = SequentialSampler(gold_data_t_medroberta)
gold_dataloader_medroberta = DataLoader(gold_data_t_medroberta, sampler=gold_sampler_medroberta, batch_size=batch_size)

test_data_t_medroberta = TensorDataset(test_inputs_medroberta, test_masks_medroberta)
test_dataloader_medroberta = DataLoader(test_data_t_medroberta, batch_size=batch_size)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.optim as optim

batch_size = 1

train_data_t_medlongformer = TensorDataset(train_inputs_medlongformer, train_masks_medlongformer, train_labels_medlongformer)
train_sampler_medlongformer = SequentialSampler(train_data_t_medlongformer)
train_dataloader_medlongformer = DataLoader(train_data_t_medlongformer, sampler=train_sampler_medlongformer, batch_size=batch_size)

validation_data_t_medlongformer = TensorDataset(val_inputs_medlongformer, val_masks_medlongformer, val_labels_medlongformer)
validation_sampler_medlongformer = SequentialSampler(validation_data_t_medlongformer)
validation_dataloader_medlongformer = DataLoader(validation_data_t_medlongformer, sampler=validation_sampler_medlongformer, batch_size=batch_size)

contradict_data_t_medlongformer = TensorDataset(contradict_inputs_medlongformer, contradict_masks_medlongformer, contradict_labels_medlongformer)
contradict_sampler_medlongformer = SequentialSampler(contradict_data_t_medlongformer)
contradict_dataloader_medlongformer = DataLoader(contradict_data_t_medlongformer, sampler=contradict_sampler_medlongformer, batch_size=batch_size)

original_data_t_medlongformer = TensorDataset(original_inputs_medlongformer, original_masks_medlongformer, original_labels_medlongformer)
original_sampler_medlongformer = SequentialSampler(original_data_t_medlongformer)
original_dataloader_medlongformer = DataLoader(original_data_t_medlongformer, sampler=original_sampler_medlongformer, batch_size=batch_size)

preserving_data_t_medlongformer = TensorDataset(preserving_inputs_medlongformer, preserving_masks_medlongformer, preserving_labels_medlongformer)
preserving_sampler_medlongformer = SequentialSampler(preserving_data_t_medlongformer)
preserving_dataloader_medlongformer = DataLoader(preserving_data_t_medlongformer, sampler=preserving_sampler_medlongformer, batch_size=batch_size)

gold_data_t_medlongformer = TensorDataset(gold_inputs_medlongformer, gold_masks_medlongformer, gold_labels_medlongformer)
gold_sampler_medlongformer = SequentialSampler(gold_data_t_medlongformer)
gold_dataloader_medlongformer = DataLoader(gold_data_t_medlongformer, sampler=gold_sampler_medlongformer, batch_size=batch_size)

test_data_t_medlongformer = TensorDataset(test_inputs_medlongformer, test_masks_medlongformer)
test_dataloader_medlongformer = DataLoader(test_data_t_medlongformer, batch_size=batch_size)


In [ ]:
train_labels_r = train_data['Label'].replace(label_mapping).values
validation_labels=validation_data['Label'].replace(label_mapping).values
contradict_labels=contradict_data['Label'].replace(label_mapping).values
original_labels=original_data['Label'].replace(label_mapping).values
preserving_labels=preserving_data['Label'].replace(label_mapping).values
gold_labels=gold_data['Label'].replace(label_mapping).values

In [ ]:
model_medbert.to(device)

In [ ]:
import numpy as np

In [ ]:
predictions_medbert_train=[]
logits_medbert_train=[]
with torch.no_grad():
    for batch in train_dataloader_medbert:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medbert(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medbert_train.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medbert_train.append(preds)
predictions_medbert_val=[]
logits_medbert_val=[]
with torch.no_grad():
    for batch in validation_dataloader_medbert:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medbert(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medbert_val.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medbert_val.append(preds)
predictions_medbert_gold=[]
logits_medbert_gold=[]
with torch.no_grad():
    for batch in gold_dataloader_medbert:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medbert(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medbert_gold.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medbert_gold.append(preds)
predictions_medbert_test=[]
logits_medbert_test=[]
with torch.no_grad():
    for batch in test_dataloader_medbert:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask = batch
        outputs = model_medbert(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medbert_test.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medbert_test.append(preds)

In [ ]:
len(predictions_medbert_train), len(predictions_medbert_val),len(predictions_medbert_test)

(1700, 200, 5500)

In [ ]:
model_medbert.to('cpu')
model_medroberta.to('cuda')

In [ ]:
predictions_medroberta_train=[]
logits_medroberta_train=[]
with torch.no_grad():
    for batch in train_dataloader_medroberta:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medroberta(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medroberta_train.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medroberta_train.append(preds)
predictions_medroberta_val=[]
logits_medroberta_val=[]
with torch.no_grad():
    for batch in validation_dataloader_medroberta:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medroberta(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medroberta_val.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medroberta_val.append(preds)
predictions_medroberta_gold=[]
logits_medroberta_gold=[]
with torch.no_grad():
    for batch in gold_dataloader_medroberta:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medroberta(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medroberta_gold.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medroberta_gold.append(preds)
predictions_medroberta_test=[]
logits_medroberta_test=[]
with torch.no_grad():
    for batch in test_dataloader_medroberta:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask = batch
        outputs = model_medroberta(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medroberta_test.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medroberta_test.append(preds)

In [ ]:
len(predictions_medroberta_train), len(predictions_medroberta_val), len(predictions_medroberta_test)

(1700, 200, 5500)

In [ ]:
model_medroberta.to('cpu')
model_medlongformer.to('cuda')

In [ ]:
predictions_medlongformer_train=[]
logits_medlongformer_train=[]
with torch.no_grad():
    for batch in train_dataloader_medlongformer:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medlongformer(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medlongformer_train.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medlongformer_train.append(preds)
predictions_medlongformer_val=[]
logits_medlongformer_val=[]
with torch.no_grad():
    for batch in validation_dataloader_medlongformer:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medlongformer(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medlongformer_val.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medlongformer_val.append(preds)
predictions_medlongformer_gold=[]
logits_medlongformer_gold=[]
with torch.no_grad():
    for batch in gold_dataloader_medlongformer:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model_medlongformer(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medlongformer_gold.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medlongformer_gold.append(preds)
predictions_medlongformer_test=[]
logits_medlongformer_test=[]
with torch.no_grad():
    for batch in test_dataloader_medlongformer:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask = batch
        outputs = model_medlongformer(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits.detach().cpu().numpy()
        logits_squeezed=logits.squeeze()
        logits_medlongformer_test.append(logits_squeezed)
        preds = np.argmax(logits, axis=1).flatten()
        predictions_medlongformer_test.append(preds)

Initializing global attention on CLS token...


In [ ]:
len(predictions_medlongformer_train), len(predictions_medlongformer_val), len(predictions_medlongformer_test)

(1700, 200, 5500)

In [ ]:
model_medlongformer.to('cpu')

In [ ]:
concatenated_logits_train = np.concatenate((logits_medbert_train, logits_medroberta_train, logits_medlongformer_train), axis=1)

In [ ]:
concatenated_logits_val = np.concatenate((logits_medbert_val, logits_medroberta_val, logits_medlongformer_val), axis=1)

In [ ]:
concatenated_logits_test = np.concatenate((logits_medbert_test, logits_medroberta_test, logits_medlongformer_test), axis=1)

In [ ]:
concatenated_logits_gold = np.concatenate((logits_medbert_gold, logits_medroberta_gold, logits_medlongformer_gold), axis=1)

In [ ]:
combined_train_predictions = np.column_stack((predictions_medbert_train, predictions_medroberta_train, predictions_medlongformer_train))

In [ ]:
combined_val_predictions = np.column_stack((predictions_medbert_val, predictions_medroberta_val, predictions_medlongformer_val))

In [ ]:
combined_test_predictions = np.column_stack((predictions_medbert_test, predictions_medroberta_test, predictions_medlongformer_test))

In [ ]:
combined_gold_predictions = np.column_stack((predictions_medbert_gold, predictions_medroberta_gold, predictions_medlongformer_gold))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
ensemble_classifier = LogisticRegression()
ensemble_classifier.fit(concatenated_logits_train, train_labels_r)
ensemble_predictions = ensemble_classifier.predict(concatenated_logits_val)
ensemble_accuracy = accuracy_score(validation_labels, ensemble_predictions)
print("Ensemble Model Accuracy:", ensemble_accuracy)


Ensemble Model Accuracy: 0.615


In [ ]:
len(ensemble_test_predictions)

5500

In [ ]:
for i in ensemble_test_predictions:
  print(i, end=', ')

0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0

In [ ]:
for i in ensemble_gold_predictions:
  print(i, end=', ')

1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0

In [ ]:
row_number = gold_data.index.get_loc('a805f174-2707-4302-a195-05153ea8523e')
print(row_number)

1665


In [ ]:
gold_data['Statement'][1665]

'Only adults can take part in the primary trial'

In [ ]:
gold_data['Label'][1665]

'Entailment'

In [ ]:
ensemble_gold_predictions[1665]

1

Ensemble learning with majority voting

In [ ]:
max_voting_train=[]
for i in range(len(combined_train_predictions)):
  zero_count=0
  one_count=1
  for j in range(2):
    if combined_train_predictions[i][j]==0:
      zero_count=zero_count+1
    else:
      one_count=one_count+1
  if zero_count>one_count:
    max_voting_train.append(0)
  else:
    max_voting_train.append(1)
len(max_voting_train)

1700

In [ ]:
correct_predictions=0
for i in range(len(max_voting_train)):
  if max_voting_train[i]==train_labels_r[i]:
    correct_predictions=correct_predictions+1
accuracy=correct_predictions*100/len(max_voting_train)
print(accuracy)

82.76470588235294


In [ ]:
max_voting_val=[]
for i in range(len(combined_val_predictions)):
  zero_count=0
  one_count=1
  for j in range(2):
    if combined_val_predictions[i][j]==0:
      zero_count=zero_count+1
    else:
      one_count=one_count+1
  if zero_count>one_count:
    max_voting_val.append(0)
  else:
    max_voting_val.append(1)
len(max_voting_val)
#max_voting_val

200

In [ ]:
correct_predictions=0
for i in range(len(max_voting_val)):
  if max_voting_val[i]==validation_labels[i]:
    correct_predictions=correct_predictions+1
accuracy=correct_predictions*100/len(max_voting_train)
print(accuracy)

6.117647058823529


In [ ]:
max_voting_test=[]
for i in range(len(combined_test_predictions)):
  zero_count=0
  one_count=1
  for j in range(2):
    if combined_test_predictions[i][j]==0:
      zero_count=zero_count+1
    else:
      one_count=one_count+1
  if zero_count>one_count:
    max_voting_test.append(0)
  else:
    max_voting_test.append(1)
len(max_voting_test)

5500

In [ ]:
for i in max_voting_test:
  print(i, end=', ')

1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1